In [1]:
from catboost import CatBoostClassifier
from scipy import stats
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import precision_score, roc_auc_score, f1_score, recall_score
from sklearn.preprocessing import StandardScaler
from fancyimpute import IterativeImputer
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.feature_selection import SelectFromModel, RFE
from sklearn.linear_model import LogisticRegression
from imblearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.model_selection import GridSearchCV
import xlsxwriter
from random import randint
random_state = 7656
from preprocessing import stds, stats, cv_preprocessing
from load_data import load_data
import os

Using TensorFlow backend.
C:\Users\nogag\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\nogag\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\nogag\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users

In [2]:
df_preprocessed, features, target_feature = load_data()


In [ ]:
["highschool_diploma", "dyslexia", "ADHD", "T1Acc1t", "T1Acc1n", "T1bias", 
 "phq1", "trait1", "state1", "PCL1", "PCL_Strict1",  "trauma_history8_1"]

In [3]:
features

['highschool_diploma',
 'dyslexia',
 'ADHD',
 'T1Acc1t',
 'T1Acc1n',
 'T1bias',
 'phq1',
 'trait1',
 'state1',
 'PCL1',
 'PCL_Strict1',
 'trauma_history8_1',
 't1_missing',
 'q6.1_INTRU',
 'q6.2_DREAM',
 'q6.3_FLASH',
 'q6.4_UPSET',
 'q6.5_PHYS',
 'q6.6_AVTHT',
 'q6.7_AVSIT',
 'q6.8_AMNES',
 'q6.9_DISINT',
 'q6.10_DTACH',
 'q6.11_NUMB',
 'q6.12_FUTRE',
 'q6.13_SLEEP',
 'q6.14_ANGER',
 'q6.15_CONC',
 'q6.16_HYPER',
 'q6.17_STRTL',
 'intrusion_score',
 'intrusion_mean',
 'intrusion_std',
 'binary_intrusion',
 'tred_score',
 'tred_mean',
 'tred_std',
 'binary_tred',
 'T1q5.1',
 'T1q5.2',
 'T1q5.3',
 'T1q5.4',
 'T1q5.5',
 'T1q5.6',
 'T1q5.7',
 'T1q5.8',
 'T1q5.9',
 'T1q3.1',
 'T1q3.2',
 'T1q3.3',
 'T1q3.4',
 'T1q3.5',
 'T1q3.6',
 'T1q3.7',
 'T1q3.8',
 'T1q3.9',
 'T1q3.10',
 'T1q3.11',
 'T1q3.12',
 'T1q3.13',
 'T1q3.14',
 'T1q3.15',
 'T1q3.16',
 'T1q3.17',
 'T1q3.18',
 'T1q3.19',
 'T1q3.20',
 'T1q2.1',
 'T1q2.2',
 'T1q2.3',
 'T1q2.4',
 'T1q2.5',
 'T1q2.6',
 'T1q2.7',
 'T1q2.8',
 'T1q2.9',
 

In [ ]:

for i in range(len(target_feature)):
    print ('\n\n\n', target_feature[i], '\n-------------------')
    X, X_out, Y, y_out = train_test_split(df_preprocessed[features], df_preprocessed[target_feature[i]],\
                                          test_size=0.25, random_state=random_state,\
                                          stratify=df_preprocessed[target_feature[i]])



    for i in [random_state, random_state-195, random_state+344, random_state-347, random_state+89]:
        X_train, X_test, y_train, y_test = train_test_split(X, Y,  random_state=i, test_size=0.2, stratify=Y)
        cv = StratifiedKFold(6, random_state=i, shuffle=True)

        X_train, X_test = cv_preprocessing(X_train, X_test, i)

        pipe = Pipeline(steps=[
        ('rfe', RFE(ExtraTreesClassifier(), 35)),
        ('classifier', CatBoostClassifier(verbose=0, random_state=i))])

        grid_params = [{
        'classifier__class_weights':[[1, 14]],#, [1, 15], [1, 30]],
        'classifier__l2_leaf_reg': [150, 250, 50],# 50],
        'classifier__depth': [6, 4],#, 9]
        }]
        clf = GridSearchCV(pipe, grid_params, cv=cv, scoring='roc_auc')
        clf.fit(X_train, y_train.values.astype(int))#, fit_params = {'classifier__early_stopping_rounds':15})
        print(f"i = {i}, roc_auc = {clf.best_score_}, params = {clf.best_params_}")
        y_pred_target = clf.best_estimator_.predict_proba(X_test)[:, 1]
        print(f"holdout i = {i}, roc_auc = {roc_auc_score(y_test.astype(int), y_pred_target)}")
        for i, j in zip(X_train.columns, clf.best_estimator_['rfe'].ranking_):
            if j == 1:
                print(i)
        print("\n\n\n")




 PCL_Strict3 
-------------------


C:\Users\nogag\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:693: RuntimeWarning: divide by zero encountered in true_divide
  return self.resid / sigma / np.sqrt(1 - hii)
C:\Users\nogag\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:773: RuntimeWarning: divide by zero encountered in true_divide
  dfbetas /= np.sqrt(self.sigma2_not_obsi[:, None])
C:\Users\nogag\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_features_to_select=35 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


i = 7656, roc_auc = 0.6954732510288065, params = {'classifier__class_weights': [1, 14], 'classifier__depth': 6, 'classifier__l2_leaf_reg': 150}
holdout i = 7656, roc_auc = 0.9160493827160493
highschool_diploma
T1bias
phq1
trait1
PCL1
trauma_history8_1
q6.1_INTRU
q6.2_DREAM
q6.6_AVTHT
q6.7_AVSIT
q6.8_AMNES
q6.9_DISINT
q6.11_NUMB
q6.12_FUTRE
q6.13_SLEEP
q6.15_CONC
q6.16_HYPER
T1q5.3
T1q5.7
T1q5.9
T1q2.1
T1q2.2
T1q2.6
T1q2.11
T1q2.12
T1q2.14
T1q2.18
T1q3.4
T1q3.5
T1q3.9
T1q3.13
T1q3.16
T1q3.17
T1q3.19
T1q3.20






C:\Users\nogag\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_features_to_select=35 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


i = 7461, roc_auc = 0.7484567901234568, params = {'classifier__class_weights': [1, 14], 'classifier__depth': 6, 'classifier__l2_leaf_reg': 150}
holdout i = 7461, roc_auc = 0.7259259259259259
highschool_diploma
T1Acc1n
T1bias
phq1
PCL1
trauma_history8_1
q6.2_DREAM
q6.3_FLASH
q6.6_AVTHT
q6.8_AMNES
q6.9_DISINT
q6.10_DTACH
q6.11_NUMB
q6.13_SLEEP
q6.15_CONC
q6.16_HYPER
q6.17_STRTL
T1q5.3
T1q5.7
T1q5.8
T1q5.9
T1q2.3
T1q2.14
T1q2.16
T1q2.18
T1q3.5
T1q3.8
T1q3.11
T1q3.12
T1q3.13
T1q3.15
T1q3.16
T1q3.17
T1q3.18
T1q3.20






...

i = 7309, roc_auc = 0.6627746719465797, params = {'classifier__class_weights': [1, 14], 'classifier__depth': 6, 'classifier__l2_leaf_reg': 250}
holdout i = 7309, roc_auc = 0.7530864197530865
highschool_diploma
ADHD
T1Acc1n
T1bias
phq1
trait1
PCL1
trauma_history8_1
q6.2_DREAM
q6.6_AVTHT
q6.7_AVSIT
q6.8_AMNES
q6.9_DISINT
q6.10_DTACH
q6.11_NUMB
q6.13_SLEEP
q6.15_CONC
q6.16_HYPER
T1q5.3
T1q5.7
T1q5.9
T1q2.1
T1q2.6
T1q2.14
T1q2.15
T1q2.16
T1q2.18
T1q3.4
T1q3.11
T1q3.12
T1q3.13
T1q3.15
T1q3.16
T1q3.17
T1q3.20






C:\Users\nogag\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_features_to_select=35 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


i = 7745, roc_auc = 0.717815824209954, params = {'classifier__class_weights': [1, 14], 'classifier__depth': 6, 'classifier__l2_leaf_reg': 50}
holdout i = 7745, roc_auc = 0.7407407407407407
T1Acc1t
T1Acc1n
T1bias
phq1
trait1
PCL1
trauma_history8_1
q6.2_DREAM
q6.6_AVTHT
q6.7_AVSIT
q6.8_AMNES
q6.9_DISINT
q6.10_DTACH
q6.11_NUMB
q6.13_SLEEP
q6.14_ANGER
q6.15_CONC
q6.17_STRTL
T1q5.3
T1q5.7
T1q5.9
T1q2.6
T1q2.11
T1q2.14
T1q2.15
T1q2.18
T1q3.4
T1q3.8
T1q3.9
T1q3.11
T1q3.15
T1q3.16
T1q3.17
T1q3.18
T1q3.20







 target_tred 
-------------------


C:\Users\nogag\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:693: RuntimeWarning: divide by zero encountered in true_divide
  return self.resid / sigma / np.sqrt(1 - hii)
C:\Users\nogag\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:773: RuntimeWarning: divide by zero encountered in true_divide
  dfbetas /= np.sqrt(self.sigma2_not_obsi[:, None])
C:\Users\nogag\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_features_to_select=35 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


i = 7656, roc_auc = 0.8081428946677174, params = {'classifier__class_weights': [1, 14], 'classifier__depth': 6, 'classifier__l2_leaf_reg': 150}
holdout i = 7656, roc_auc = 0.7502347417840376
T1Acc1t
T1Acc1n
T1bias
phq1
trait1
trauma_history8_1
q6.1_INTRU
q6.2_DREAM
q6.3_FLASH
q6.4_UPSET
q6.6_AVTHT
q6.9_DISINT
q6.10_DTACH
q6.11_NUMB
q6.12_FUTRE
q6.13_SLEEP
q6.15_CONC
q6.16_HYPER
q6.17_STRTL
T1q5.5
T1q5.6
T1q5.9
T1q2.1
T1q2.2
T1q2.3
T1q2.17
T1q3.2
T1q3.5
T1q3.7
T1q3.9
T1q3.11
T1q3.13
T1q3.17
T1q3.18
T1q3.20






In [3]:
100/7

14.285714285714286